# core

> Fill in a module description here

In [31]:
#| default_exp google.creds

In [32]:
#| hide
from nbdev.showdoc import *


In [33]:
#| exporti

import os

from google.oauth2.credentials import Credentials
from google.auth.transport.requests import Request
from google_auth_oauthlib.flow import InstalledAppFlow

ModuleNotFoundError: No module named 'google.oauth2'

In [ ]:
#|export

def get_creds(token_file_path="env_token.json", credentials_file_path="env_creds.json"):
    """generate creds here, https://console.cloud.google.com/apis/credentials/oauthclient/214272795931-8l2hleihje6d14a0ltali43dpu26jjob.apps.googleusercontent.com?authuser=2&project=plated-ensign-355620
    for a new implementation you'll need to
    1. set up a project at console.cloud.google.com,
    2. enable (turn on) the appropriate APIs for that project
    3. configure OAuth access (your python script will use an Oauth token to authenticate, so your project needs the OAuth screen)
    4. download the creds file and store it as env_creds.json
    """

    # If modifying these scopes, delete the file token.json.
    SCOPES = [
        "https://www.googleapis.com/auth/drive",
        "https://www.googleapis.com/auth/drive.file",
        "https://www.googleapis.com/auth/drive.metadata.readonly",
        "https://www.googleapis.com/auth/drive.readonly",
    ]

    creds = None
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if token_file_path and os.path.exists(token_file_path):
        creds = Credentials.from_authorized_user_file(token_file_path, SCOPES)

    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            # if creds token can be refreshed, do it.
            creds.refresh(Request())
        else:
            # If there are no (valid) credentials available, run the Oauth flow
            flow = InstalledAppFlow.from_client_secrets_file(
                credentials_file_path, SCOPES
            )
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open(token_file_path, "w") as token:
            token.write(creds.to_json())
    return creds


creds = get_creds()

In [ ]:
# in this example we assume the client secret information is loaded into the environment as GDOC_KEY using a .env file
# we must generate an env_token.json file for authenticating queries

from dotenv import load_dotenv

load_dotenv('.env')

client_secret_info=json.loads(os.environ['GDOC_KEY'])

if not os.path.exists('env_token.json'):
    generate_token_from_oauth_with_client_info(client_secret_info = client_secret_info)


#### sample implementation of get_creds

In [ ]:
#| export

def generate_creds_object(
        token_file_path: str = None,
        token_info: dict = None,
        client_secret_path: str = None,
        client_secret_info: dict = None):
    """generate creds object from token file"""
    
    creds = None

    if token_file_path:
        creds = Credentials.from_authorized_user_file(token_file_path)

    if token_info:
        creds = Credentials.from_authorized_user_info(token_info)

    if creds and creds.valid:
        return creds

    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
        return creds

    # If there are no (valid) credentials available, let the user log in.
    return generate_token_from_oauth_with_client_info(token_file_path=token_file_path or 'env_token.json',
                                                      client_secret_path=client_secret_path,
                                                      client_secret_info=client_secret_info)


In [ ]:
from dotenv import load_dotenv

load_dotenv('.env')

client_secret_info=json.loads(os.environ['GDOC_KEY'])

token_info = json.loads(os.environ['GDOC_TOKEN'])

generate_creds_object(client_secret_info=client_secret_info, token_info = token_info)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()